## <div align="center"> TUGAS LAB IF540 MACHINE LEARNING </div>
## <div align="center"> Semester Genap 2022/2023 </div>
### <div align="center"> WEEK [02] : [DATA PREPROCESSING] </div>


### Header

In [60]:
import datetime
import uuid    

# Fill in your name and NIM
myName = "Gempar Bambang Godata"
myNIM = "00000067643"

myDate = datetime.datetime.now()
myDevice = str(uuid.uuid1())

# Header
print("Name: \t\t{}".format(myName))
print("NIM: \t\t{}".format(myNIM))
print("Start: \t\t{}".format(myDate))
print("Device ID: \t{}".format(myDevice))

Name: 		Gempar Bambang Godata
NIM: 		00000067643
Start: 		2023-02-17 05:13:10.436982
Device ID: 	190a4ca0-ae47-11ed-9950-c424df8cd950


### Deskripsi Singkat Dataset:

tulis deskripsi singkat dataset yang Anda gunakan


---

### Dataset yang diambil dari repositori:

1. Indonesia's Trending YouTube Video Statistics – sumber : https://www.kaggle.com/datasets/syahrulhamdani/indonesias-trending-youtube-video-statistics?select=trending.csv
2. Basic Computer Data – sumber : https://www.kaggle.com/datasets/kingburrito666/basic-computer-data-set

### Hasil kerja

### Dealing with Missing Data 

#### Lets create a simple data from a CSV (comma-separated values) file to get a better graps of the problem. 

In [2]:
from IPython.display import Image
%matplotlib inline

In [3]:
import pandas as pd
from io import StringIO
import sys

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


#### Working with numpy array, it can sometimes be more convenient to preprocess data using pandas DataFrame

In [4]:
# Access the underlying numpy array via the 'values' attribute

df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

### Eliminating Samples of Features With Missing Values

#### One of the easiest ways to deal with missing data is to simply remove the corresponding features (columns) or samples (rows) from the dataset entirely; rows with missing values can be easily dropped via the dropna methods :

In [5]:
# Remove rows that contain missing values
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


#### Using the sum method, we can return the number of missing values per columns as follows: 

In [6]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

#### Although, scikit-learn was developed for Similarly, we can drop columns that have at least one NaN in any row by setting the axis argument to 1 

In [7]:
# Remove columns that contain missing values
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


#### The dropna method supports several additional parameters that can come in handy :

In [8]:
# only drop rows where all columns are NaN
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [9]:
# drop rows that have fewer than 3 real values
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [10]:
# only drop rows where NaN appear in specific columns (here: 'C')
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### Imputing Missing Values

#### A convenient way to achieve this is by using the Imputer class from scikit-learn as shown in the following code :

In [11]:
# again: our original array
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [12]:
# impute missing values via the column mean

from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [13]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


### Handling Categorical Data

#### Lets create a new data frame to illustrate the problem (different between data type)

In [14]:
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                  ['red', 'L', 13.5, 'class1'],
                  ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


#### Make sure that the learning algorithms interprets the ordinal features correctly, we need to convert the categorical string values into integers. Thus, we have to define the mapping manually. In the example, lets assume that we know the difference between features

In [15]:
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


#### If we want to transform the integer values back to the ordinal string representation at a later stage, we can simply define a reverse mapping dictionary inv_size_mapping. That can be used via the pandas map method on the transformed feature column similar to the size_mapping dictionary that we used previously. 

In [16]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### Encoding Class Label

#### We need to remember that class labels are not ordinal, and it doesn’t matter which integer number we assign to a particular string label. Thus, we can simply enumerate the class labels starting at 0 :

In [17]:
import numpy as np

# create a mapping dict
# to convert class labels from strings to integers
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

#### Next we can use the mapping dictionary to transform the class labels into integers : 

In [18]:
# to convert class labels from strings to integers
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


#### We can reverse the key – value pairs in the mapping dictionary as follows to map the converted class labels back to the original string representation :

In [19]:
# reverse the class label mapping
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


#### There is a convenient LabelEncoder class directly implemented in scikit-learn to achieve the same :

In [20]:
from sklearn.preprocessing import LabelEncoder

# Label encoding with sklearn's LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

#### Note that the fit_transform method is just a shortcut for calling fit and transform separately and we can use the inverse_transform method to transform the integer class labels back into their original string representation : 

In [21]:
# reverse mapping
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

### Performing one-hot encoding on nominal features 

#### We used the convenient LabelEncoder class to encode the string labels into integers. It may appear that we could use a similar approach to transform the nomimal color columns for our dataset, as follows :

In [22]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

#### The solution technique called one-hot encoding. The idea behind this approach is to create a new dummy feature for each unique value in the nominal feature columns. Here, we would convert the color feature into three new features (blue, green, red). Binary values can then be used to indicate the particular color of a sample. To perform this transformation, we can use the OneHotEncoder that is implemented in the scikit-learn.preprocessing module : 

In [23]:
from sklearn.preprocessing import OneHotEncoder

x = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [24]:
from sklearn.compose import ColumnTransformer

X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([ ('onehot', OneHotEncoder(), [0]),
                              ('nothing', 'passthrough', [1, 2])])
c_transf.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

#### Applied on a DataFrame, the get_dummies (implemented in pandas) method will only convert string columns and leave all other columns unchanged : 

In [25]:
# one-hot encoding via pandas
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [26]:
#multicollinearity guard in get_dummies
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


In [27]:
# multicollinearity guard for the OneHotEncoder
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([ ('onehot', color_ohe, [0]),
                              ('nothing', 'passthrough', [1, 2])])
c_transf.fit_transform(X).astype(float)

array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

### Partitioning Dataset in Training and Test Sets

#### In this section, we will prepare a new dataset, the Wine Dataset. After we have preprocessed the dataset, we will explore different techniques for feature selection to reduce the dimensionality of a dataset. Using the pandas library, we will directly read in the open source Wine dataset from the UCI machine learning repository :

In [28]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)

# if the Wine dataset is temporarily unavailable from the
# UCI machine learning repository, un-comment the following line
# of code to load the dataset from a local path:

# df wine = pd.read csv('wine.data', header=None)


df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnegium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', '0D280/0D315 of diluted wines',
                   'Proline']

print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnegium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,0D280/0D315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


#### A convenient way to randomly partition this dataset into a separate test and training dataset is to use the train_test_split function from scikit-learn cross_validation submodule. First, we assigned the NumPy array representation of feature columns 1-13 to the variable X, and we assigned the class labels from the first column to the variable y. Then, we used the train_test_split function to randomly split X and y into separate training and test datasets. By setting test_size=0.3 we assigned 30 percent of the wine samples to X_test and y_test, and the remaining 70 percent of the samples were assigned to X_train and y_train, respectively.

In [29]:
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test =\
    train_test_split (X, y,
                      test_size=0.3,
                      random_state=0,
                      stratify=y)

### Bringing Features Onto the Same Scale 

#### Normalization : refers to the rescaling of the features to a range of [0, 1] which is a special case of mix-max scaling.The min-max scaling procedure is implemented in scikit-learn and can be used as follows : 

In [30]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

#### Standardization : we center the feature columns at mean 0 with standard deviation 1 so that the feature columns take the norm of a normal distribution which makes it easier to learn the weight. Similar to MinMaxScaler, scikit-learn also implements a class for standardization : 

In [31]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

#### Here, a visual example of normalization and standardization : 

In [32]:
ex = np.array([0, 1, 2, 3, 4, 5])

print('standardized:', (ex - ex.mean()) / ex.std())

# normalize

print('normalized:', (ex - ex.min()) / (ex.max() - ex.min()))

standardized: [-1.46385011 -0.87831007 -0.29277002  0.29277002  0.87831007  1.46385011]
normalized: [0.  0.2 0.4 0.6 0.8 1. ]


## Challange Dataset Sendiri

### Dataset 1
### Handling Numerical Data Null
#### Mengimpor libraries yang dibutuhkan

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud
from IPython.display import Image
from io import StringIO
import sys
%matplotlib inline

#### Mengimpor file dataset 1

In [34]:
df = pd.read_csv('trending.csv')
df

C:\Users\user\AppData\Local\Temp\ipykernel_13012\2341679072.py:1: DtypeWarning: Columns (0,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('trending.csv')


,video_id,publish_time,channel_id,title,description,thumbnail_url,thumbnail_width,thumbnail_height,channel_name,tags,...,caption,license_status,allowed_region,blocked_region,view,like,dislike,favorite,comment,trending_time
0,NaN,2021-02-05T09:00:34Z,UCEf_Bc-KVd7onSeifS3py9g,aespa 에스파 'Forever (약속)' MV,"aespa's new single ""Forever"" is out!\nListen a...",NaN,NaN,NaN,SMTOWN,NaN,...,True,True,NaN,NaN,7806846.0,857902.0,24078.0,0,94961.0,2021-02-07 05:46:51.832614+00:00
1,NaN,2021-02-04T15:54:08Z,UCYEa4_MC7OFjcWrO6SO-u8g,[FULL] Siapa di Balik Kudeta AHY? | Dua Sisi t...,"Jakarta, https://www.tvonenews.com - Ketua umu...",NaN,NaN,NaN,Indonesia Lawyers Club,"['tvone', 'tv one', 'tvonenews', 'indonesia la...",...,False,True,NaN,NaN,885038.0,6765.0,1263.0,0,6365.0,2021-02-07 05:46:51.832649+00:00
2,NaN,2021-02-06T03:00:22Z,UCu12RxQjRQyh79YBYvEDkOg,CABRIOLET CHALLENGE: TANTANGAN MENGGODA (7/12),Road Party Season 2: Cabriolet Challenge\n\nEp...,NaN,NaN,NaN,Motomobi,"['MotoMobi', 'Mobil', 'motor', 'review', 'indo...",...,False,True,NaN,NaN,889708.0,47895.0,532.0,0,8785.0,2021-02-07 05:46:51.832664+00:00
3,NaN,2021-02-05T20:26:08Z,UCCuzDCoI3EUOo_nhCj4noSw,With Windah Basudara & Hans,Join this channel to get access to perks:\nhtt...,NaN,NaN,NaN,yb,NaN,...,False,True,NaN,NaN,1006854.0,91973.0,3967.0,0,12957.0,2021-02-07 05:46:51.832678+00:00
4,NaN,2021-02-03T23:14:54Z,UC14UlmYlSNiQCBe9Eookf_A,🤯 LATE COMEBACK DRAMA! | HIGHLIGHTS | Granada ...,"With just two minutes to play, Barça looked to...",NaN,NaN,NaN,FC Barcelona,"['FC Barcelona', 'برشلونة،', 'Fútbol', 'FUTBOL...",...,False,True,NaN,NaN,6275035.0,218131.0,4289.0,0,12799.0,2021-02-07 05:46:51.832730+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135344,JFm7YquassM,2023-02-01T11:03:25Z,UCn8gpJ5hjZcFkiWk4rqAUXw,RASANYA GIMANA???🧑‍🍳| MASAK RAWON GAK PAKE RES...,"HALO GUYS, OHAYOU KONNICHIWA KONBANWA!!! AKU Y...",https://i.ytimg.com/vi/JFm7YquassM/hqdefault.jpg,480.0,360.0,YUSUKE【wasedaboys】,NaN,...,False,True,NaN,NaN,463329.0,21886.0,NaN,0,983.0,2023-02-13 06:00:46.713638+00:00
135345,1UrfAseM5xU,2023-01-30T10:19:31Z,UCknKkn8IPXkMlV8n_ajX7Dw,KONDANGAN AUTO JADI MAK COMBLANG?!!! 😱 … WEDDI...,Haii guys thanks for watching 🙆🏻‍♀️💜\nSemoga k...,https://i.ytimg.com/vi/1UrfAseM5xU/hqdefault.jpg,480.0,360.0,Adira Sahara,NaN,...,False,True,NaN,NaN,647444.0,10634.0,NaN,0,179.0,2023-02-13 06:00:46.713645+00:00
135346,tap8yITU_Vs,2023-02-01T18:00:27Z,UCbyVnlQdFIcdViuoPIPK68A,"Preview, Tes Kamera & Harga Samsung Galaxy S23...","Ini adalah Galaxy S23 Ultra, smartphone flagsh...",https://i.ytimg.com/vi/tap8yITU_Vs/hqdefault.jpg,480.0,360.0,Jagat Review,"['Samsung', 'smartphone', 'Android', 'flagship...",...,False,True,NaN,NaN,206219.0,3318.0,NaN,0,777.0,2023-02-13 06:00:46.713650+00:00
135347,REkW4ADU9_A,2023-01-29T14:21:15Z,UC34xi8JB1AKSotPUKHywbaA,Leo Rolly Carnando/Daniel Martin (INA) VS He J...,Straight Game Akhirnya Indonesia Juara!\nPerju...,https://i.ytimg.com/vi/REkW4ADU9_A/hqdefault.jpg,480.0,360.0,MNCTV OFFICIAL,"['mnctv', 'mnctv official', 'sinetron', 'sinet...",...,False,True,NaN,NaN,946060.0,6864.0,NaN,0,459.0,2023-02-13 06:00:46.713656+00:00


#### Preprocessing menggunakan numpy array

In [35]:
# Access the underlying numpy array via the 'values' attribute

df.values

array([[nan, '2021-02-05T09:00:34Z', 'UCEf_Bc-KVd7onSeifS3py9g', ..., 0,
        94961.0, '2021-02-07 05:46:51.832614+00:00'],
       [nan, '2021-02-04T15:54:08Z', 'UCYEa4_MC7OFjcWrO6SO-u8g', ..., 0,
        6365.0, '2021-02-07 05:46:51.832649+00:00'],
       [nan, '2021-02-06T03:00:22Z', 'UCu12RxQjRQyh79YBYvEDkOg', ..., 0,
        8785.0, '2021-02-07 05:46:51.832664+00:00'],
       ...,
       ['tap8yITU_Vs', '2023-02-01T18:00:27Z',
        'UCbyVnlQdFIcdViuoPIPK68A', ..., 0, 777.0,
        '2023-02-13 06:00:46.713650+00:00'],
       ['REkW4ADU9_A', '2023-01-29T14:21:15Z',
        'UC34xi8JB1AKSotPUKHywbaA', ..., 0, 459.0,
        '2023-02-13 06:00:46.713656+00:00'],
       ['kl0cVcmjki4', '2023-01-31T04:09:22Z',
        'UC8rHUssNDJ9YEjfduAqJBmw', ..., 0, 2142.0,
        '2023-02-13 06:00:46.713662+00:00']], dtype=object)

### Menghilangkan sampel Features dengan nilai yang hilang
#### Menghapus baris yang berisi nilai yang hilang

In [36]:
df.dropna(axis=0)

,video_id,publish_time,channel_id,title,description,thumbnail_url,thumbnail_width,thumbnail_height,channel_name,tags,...,caption,license_status,allowed_region,blocked_region,view,like,dislike,favorite,comment,trending_time


#### Dikarenakan setiap baris memiliki nilai null, maka kita harus mengembalikan data dengan menggunakan metode sum. Kita dapat mengembalikan jumlah nilai yang hilang per kolom sebagai berikut:

In [37]:
df.isnull().sum()

video_id              53122
publish_time              0
channel_id                0
title                     0
description            4888
thumbnail_url         53122
thumbnail_width       53122
thumbnail_height      53122
channel_name              0
tags                  18408
category_id               0
live_status               0
local_title               0
local_description      4888
duration                  0
dimension                 0
definition                0
caption                   0
license_status            0
allowed_region       130855
blocked_region       130082
view                     10
like                   1109
dislike               81719
favorite                  0
comment                 445
trending_time             0
dtype: int64

#### Remove columns that contain missing values

In [38]:
df.dropna(axis=1)

,publish_time,channel_id,title,channel_name,category_id,live_status,local_title,duration,dimension,definition,caption,license_status,favorite,trending_time
0,2021-02-05T09:00:34Z,UCEf_Bc-KVd7onSeifS3py9g,aespa 에스파 'Forever (약속)' MV,SMTOWN,10,none,aespa 에스파 'Forever (약속)' MV,PT5M7S,2d,hd,True,True,0,2021-02-07 05:46:51.832614+00:00
1,2021-02-04T15:54:08Z,UCYEa4_MC7OFjcWrO6SO-u8g,[FULL] Siapa di Balik Kudeta AHY? | Dua Sisi t...,Indonesia Lawyers Club,25,none,[FULL] Siapa di Balik Kudeta AHY? | Dua Sisi t...,PT42M30S,2d,hd,False,True,0,2021-02-07 05:46:51.832649+00:00
2,2021-02-06T03:00:22Z,UCu12RxQjRQyh79YBYvEDkOg,CABRIOLET CHALLENGE: TANTANGAN MENGGODA (7/12),Motomobi,2,none,CABRIOLET CHALLENGE: TANTANGAN MENGGODA (7/12),PT46M43S,2d,hd,False,True,0,2021-02-07 05:46:51.832664+00:00
3,2021-02-05T20:26:08Z,UCCuzDCoI3EUOo_nhCj4noSw,With Windah Basudara & Hans,yb,20,none,With Windah Basudara & Hans,PT45M59S,2d,hd,False,True,0,2021-02-07 05:46:51.832678+00:00
4,2021-02-03T23:14:54Z,UC14UlmYlSNiQCBe9Eookf_A,🤯 LATE COMEBACK DRAMA! | HIGHLIGHTS | Granada ...,FC Barcelona,17,none,🤯 LATE COMEBACK DRAMA! | HIGHLIGHTS | Granada ...,PT5M12S,2d,hd,False,True,0,2021-02-07 05:46:51.832730+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135344,2023-02-01T11:03:25Z,UCn8gpJ5hjZcFkiWk4rqAUXw,RASANYA GIMANA???🧑‍🍳| MASAK RAWON GAK PAKE RES...,YUSUKE【wasedaboys】,22,none,RASANYA GIMANA???🧑‍🍳| MASAK RAWON GAK PAKE RES...,PT25M3S,2d,hd,False,True,0,2023-02-13 06:00:46.713638+00:00
135345,2023-01-30T10:19:31Z,UCknKkn8IPXkMlV8n_ajX7Dw,KONDANGAN AUTO JADI MAK COMBLANG?!!! 😱 … WEDDI...,Adira Sahara,22,none,KONDANGAN AUTO JADI MAK COMBLANG?!!! 😱 … WEDDI...,PT10M36S,2d,hd,False,True,0,2023-02-13 06:00:46.713645+00:00
135346,2023-02-01T18:00:27Z,UCbyVnlQdFIcdViuoPIPK68A,"Preview, Tes Kamera & Harga Samsung Galaxy S23...",Jagat Review,28,none,"Preview, Tes Kamera & Harga Samsung Galaxy S23...",PT21M33S,2d,hd,False,True,0,2023-02-13 06:00:46.713650+00:00
135347,2023-01-29T14:21:15Z,UC34xi8JB1AKSotPUKHywbaA,Leo Rolly Carnando/Daniel Martin (INA) VS He J...,MNCTV OFFICIAL,24,none,Leo Rolly Carnando/Daniel Martin (INA) VS He J...,PT12M4S,2d,hd,False,True,0,2023-02-13 06:00:46.713656+00:00


#### Hanya membuang baris tempat NaN muncul di kolom tertentu (di sini: 'like', and 'dislike')

In [39]:
df.dropna(subset=['like','dislike'])

,video_id,publish_time,channel_id,title,description,thumbnail_url,thumbnail_width,thumbnail_height,channel_name,tags,...,caption,license_status,allowed_region,blocked_region,view,like,dislike,favorite,comment,trending_time
0,NaN,2021-02-05T09:00:34Z,UCEf_Bc-KVd7onSeifS3py9g,aespa 에스파 'Forever (약속)' MV,"aespa's new single ""Forever"" is out!\nListen a...",NaN,NaN,NaN,SMTOWN,NaN,...,True,True,NaN,NaN,7806846.0,857902.0,24078.0,0,94961.0,2021-02-07 05:46:51.832614+00:00
1,NaN,2021-02-04T15:54:08Z,UCYEa4_MC7OFjcWrO6SO-u8g,[FULL] Siapa di Balik Kudeta AHY? | Dua Sisi t...,"Jakarta, https://www.tvonenews.com - Ketua umu...",NaN,NaN,NaN,Indonesia Lawyers Club,"['tvone', 'tv one', 'tvonenews', 'indonesia la...",...,False,True,NaN,NaN,885038.0,6765.0,1263.0,0,6365.0,2021-02-07 05:46:51.832649+00:00
2,NaN,2021-02-06T03:00:22Z,UCu12RxQjRQyh79YBYvEDkOg,CABRIOLET CHALLENGE: TANTANGAN MENGGODA (7/12),Road Party Season 2: Cabriolet Challenge\n\nEp...,NaN,NaN,NaN,Motomobi,"['MotoMobi', 'Mobil', 'motor', 'review', 'indo...",...,False,True,NaN,NaN,889708.0,47895.0,532.0,0,8785.0,2021-02-07 05:46:51.832664+00:00
3,NaN,2021-02-05T20:26:08Z,UCCuzDCoI3EUOo_nhCj4noSw,With Windah Basudara & Hans,Join this channel to get access to perks:\nhtt...,NaN,NaN,NaN,yb,NaN,...,False,True,NaN,NaN,1006854.0,91973.0,3967.0,0,12957.0,2021-02-07 05:46:51.832678+00:00
4,NaN,2021-02-03T23:14:54Z,UC14UlmYlSNiQCBe9Eookf_A,🤯 LATE COMEBACK DRAMA! | HIGHLIGHTS | Granada ...,"With just two minutes to play, Barça looked to...",NaN,NaN,NaN,FC Barcelona,"['FC Barcelona', 'برشلونة،', 'Fútbol', 'FUTBOL...",...,False,True,NaN,NaN,6275035.0,218131.0,4289.0,0,12799.0,2021-02-07 05:46:51.832730+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54254,CkA1E5lhk_g,2021-11-30T03:00:08Z,UC5gxP-2QqIh_09djvlm9Xcg,BRAVE DAD RESCUES HIS FAMILY 🔋💚,"Hey Alan Army, this is Alan Chikin Chow! This ...",https://i.ytimg.com/vi/CkA1E5lhk_g/hqdefault.jpg,480.0,360.0,Alan Chikin Chow,"['alan chikin chow', 'alan chikin chow tiktok'...",...,False,False,NaN,NaN,31444871.0,1232586.0,104944.0,0,896.0,2021-12-13 06:01:24.993371+00:00
54255,28xbJGtkk4o,2021-12-01T12:52:57Z,UCIc9Kx9naOGjmgYWnDDjs6A,"Teman yang selalu ""ada""",#shorts\nEMAIL :\nfaizsadad.inq@gmail.com\n\nB...,https://i.ytimg.com/vi/28xbJGtkk4o/hqdefault.jpg,480.0,360.0,MAY I SEE,"['comedy', 'mayisee', 'may', 'see', 'pubg', 'f...",...,False,True,NaN,NaN,3276238.0,146154.0,6158.0,0,888.0,2021-12-13 06:01:24.993393+00:00
54256,7cbFbmfLTGY,2021-12-02T14:00:14Z,UCZIEmkvbSV8wBVWnP4wgftA,Kisah Tanah Jawa X Jurnal Risa - Gua Jepang,Selamat menikmati tayangan Kisah Tanah Jawa se...,https://i.ytimg.com/vi/7cbFbmfLTGY/hqdefault.jpg,480.0,360.0,Kisah Tanah Jawa,"['#KisahTanahJawa', '#OmHao', '#Jawa', '#Pulau...",...,False,True,NaN,NaN,874591.0,47517.0,444.0,0,2636.0,2021-12-13 06:01:24.993414+00:00
54257,IhYSCNeRX9I,2021-12-11T07:19:12Z,UCbZV9C9RPhFdIkNRoHqi6aw,BATHROOM TOUR CANGGIH RUMAH BARU TITI DAN TIAN,Seru banget bisa lihat-lihat langsung kebutuha...,https://i.ytimg.com/vi/IhYSCNeRX9I/hqdefault.jpg,480.0,360.0,Titi dan Tian,"['titi dan tian', 'vlog artis', 'titi kamal', ...",...,False,True,NaN,NaN,174208.0,6799.0,82.0,0,459.0,2021-12-13 06:01:24.993435+00:00


### Dataset 2
### Handling Categorical Data 
#### Mengimpor Dataset 2

In [40]:
import pandas as pd

df = pd.read_csv('Computers.csv')

df

,Unnamed: 0,price,speed,hd,ram,screen,premium,ads,trend
0,1,1499,25,80,4Gb,14,yes,94,1
1,2,1795,33,85,2Gb,14,yes,94,1
2,3,1595,25,170,4Gb,15,yes,94,1
3,4,1849,25,170,8Gb,14,no,94,1
4,5,3295,33,340,16Gb,14,yes,94,1
...,...,...,...,...,...,...,...,...,...
6254,6255,1690,100,528,8Gb,15,yes,39,35
6255,6256,2223,66,850,16Gb,15,yes,39,35
6256,6257,2654,100,1200,24Gb,15,yes,39,35
6257,6258,2195,100,850,16Gb,15,yes,39,35


#### Make sure that the learning algorithms interprets the ordinal features correctly, we need to convert the categorical string values into integers. Thus, we have to define the mapping manually. In the example, lets assume that we know the difference between features. 

In [41]:
subs_mapping = {'yes': 2,
                'no': 1}

df['premium'] = df['premium'].map(subs_mapping)
df

,Unnamed: 0,price,speed,hd,ram,screen,premium,ads,trend
0,1,1499,25,80,4Gb,14,2,94,1
1,2,1795,33,85,2Gb,14,2,94,1
2,3,1595,25,170,4Gb,15,2,94,1
3,4,1849,25,170,8Gb,14,1,94,1
4,5,3295,33,340,16Gb,14,2,94,1
...,...,...,...,...,...,...,...,...,...
6254,6255,1690,100,528,8Gb,15,2,39,35
6255,6256,2223,66,850,16Gb,15,2,39,35
6256,6257,2654,100,1200,24Gb,15,2,39,35
6257,6258,2195,100,850,16Gb,15,2,39,35


#### If we want to transform the integer values back to the ordinal string representation at a later stage, we can simply define a reverse mapping dictionary inv_class_mapping. That can be used via the pandas map method on the transformed feature column similar to the class_mapping dictionary that we used previously. 

In [42]:
inv_subs_mapping = {v: k for k, v in subs_mapping.items()}
df['premium'].map(inv_subs_mapping)

0       yes
1       yes
2       yes
3        no
4       yes
       ... 
6254    yes
6255    yes
6256    yes
6257    yes
6258    yes
Name: premium, Length: 6259, dtype: object

### Encoding Class Label

In [43]:
import numpy as np

# create a mapping dict
# to convert ram labels from strings to integers
ram_mapping = {label: idx for idx, label in enumerate(np.unique(df['ram']))}
ram_mapping

{'16Gb': 0, '24Gb': 1, '2Gb': 2, '32': 3, '32Gb': 4, '4Gb': 5, '8Gb': 6}

#### Next we can use the mapping dictionary to transform the class labels into integers : 

In [44]:
# to convert departure_time labels from strings to integers
df['ram'] = df['ram'].map(ram_mapping)
df

,Unnamed: 0,price,speed,hd,ram,screen,premium,ads,trend
0,1,1499,25,80,5,14,2,94,1
1,2,1795,33,85,2,14,2,94,1
2,3,1595,25,170,5,15,2,94,1
3,4,1849,25,170,6,14,1,94,1
4,5,3295,33,340,0,14,2,94,1
...,...,...,...,...,...,...,...,...,...
6254,6255,1690,100,528,6,15,2,39,35
6255,6256,2223,66,850,0,15,2,39,35
6256,6257,2654,100,1200,1,15,2,39,35
6257,6258,2195,100,850,0,15,2,39,35


#### We can reverse the key – value pairs in the mapping dictionary as follows to map the converted class labels back to the original string representation :

In [45]:
# reverse the departure_time label mapping
inv_ram_mapping = {v: k for k, v in ram_mapping.items()}
df['ram'] = df['ram'].map(inv_ram_mapping)
df

,Unnamed: 0,price,speed,hd,ram,screen,premium,ads,trend
0,1,1499,25,80,4Gb,14,2,94,1
1,2,1795,33,85,2Gb,14,2,94,1
2,3,1595,25,170,4Gb,15,2,94,1
3,4,1849,25,170,8Gb,14,1,94,1
4,5,3295,33,340,16Gb,14,2,94,1
...,...,...,...,...,...,...,...,...,...
6254,6255,1690,100,528,8Gb,15,2,39,35
6255,6256,2223,66,850,16Gb,15,2,39,35
6256,6257,2654,100,1200,24Gb,15,2,39,35
6257,6258,2195,100,850,16Gb,15,2,39,35


#### There is a convenient LabelEncoder class directly implemented in scikit-learn to achieve the same :

In [46]:
from sklearn.preprocessing import LabelEncoder

# Label encoding with sklearn's LabelEncoder
ram_le = LabelEncoder()
y = ram_le.fit_transform(df['ram'].values)
y

array([5, 2, 5, ..., 1, 0, 0])

#### Note that the fit_transform method is just a shortcut for calling fit and transform separately and we can use the inverse_transform method to transform the integer class labels back into their original string representation : 

In [47]:
# reverse mapping
ram_le.inverse_transform(y)

array(['4Gb', '2Gb', '4Gb', ..., '24Gb', '16Gb', '16Gb'], dtype=object)

### Performing one-hot encoding on nominal features 

#### We used the convenient LabelEncoder class to encode the string labels into integers. It may appear that we could use a similar approach to transform the nomimal color columns for our dataset, as follows :

In [48]:
X = df[['ram', 'premium', 'price']].values
computer_le = LabelEncoder()
X[:, 0] = computer_le.fit_transform(X[:, 0])
X

array([[5, 2, 1499],
       [2, 2, 1795],
       [5, 2, 1595],
       ...,
       [1, 2, 2654],
       [0, 2, 2195],
       [0, 2, 2490]], dtype=object)

#### The solution technique called one-hot encoding. The idea behind this approach is to create a new dummy feature for each unique value in the nominal feature columns. Here, we would convert the ram feature into 7 new features type of ram. Binary values can then be used to indicate the particular color of a sample. To perform this transformation, we can use the OneHotEncoder that is implemented in the scikit-learn.preprocessing module : 

In [49]:
from sklearn.preprocessing import OneHotEncoder

x = df[['ram', 'premium', 'price']].values
computer_le = OneHotEncoder()
computer_le.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [50]:
from sklearn.compose import ColumnTransformer

X = df[['ram', 'premium', 'price']].values
c_transf = ColumnTransformer([ ('onehot', OneHotEncoder(), [0]),
                              ('nothing', 'passthrough', [1, 2])])
c_transf.fit_transform(X).astype(float)

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        1.499e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 0.000e+00, 2.000e+00,
        1.795e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        1.595e+03],
       ...,
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        2.654e+03],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        2.195e+03],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        2.490e+03]])

#### Applied on a DataFrame, the get_dummies (implemented in pandas) method will only convert string columns and leave all other columns unchanged : 

In [51]:
# one-hot encoding via pandas
pd.get_dummies(df[['ram', 'premium', 'price']])

,premium,price,ram_16Gb,ram_24Gb,ram_2Gb,ram_32,ram_32Gb,ram_4Gb,ram_8Gb
0,2,1499,0,0,0,0,0,1,0
1,2,1795,0,0,1,0,0,0,0
2,2,1595,0,0,0,0,0,1,0
3,1,1849,0,0,0,0,0,0,1
4,2,3295,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
6254,2,1690,0,0,0,0,0,0,1
6255,2,2223,1,0,0,0,0,0,0
6256,2,2654,0,1,0,0,0,0,0
6257,2,2195,1,0,0,0,0,0,0


In [52]:
#multicollinearity guard in get_dummies
pd.get_dummies(df[['ram', 'premium', 'price']], drop_first=True)

,premium,price,ram_24Gb,ram_2Gb,ram_32,ram_32Gb,ram_4Gb,ram_8Gb
0,2,1499,0,0,0,0,1,0
1,2,1795,0,1,0,0,0,0
2,2,1595,0,0,0,0,1,0
3,1,1849,0,0,0,0,0,1
4,2,3295,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
6254,2,1690,0,0,0,0,0,1
6255,2,2223,0,0,0,0,0,0
6256,2,2654,1,0,0,0,0,0
6257,2,2195,0,0,0,0,0,0


In [53]:
# multicollinearity guard for the OneHotEncoder
computer_le = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([ ('onehot', computer_le, [0]),
                              ('nothing', 'passthrough', [1, 2])])
c_transf.fit_transform(X).astype(float)

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        1.499e+03],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        1.795e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        1.595e+03],
       ...,
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        2.654e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        2.195e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        2.490e+03]])

### Kesimpulan

Beberapa kesimpulan yang dapat ditarik dari hasil kerja melakukan data preprocessing pada machine learning adalah:
1. Preprocessing sangat penting untuk melatih model dengan data berkualitas tinggi. Proses pembersihan, transformasi, dan penskalaan data termasuk dalam lapisan ini.
2. Data yang tidak akurat, duplikat, atau hilang dihapus dalam fase data cleaning pada dataset pertama. Proses ini sangat penting untuk memastikan data yang andal dan tidak bias digunakan saat dijadikan training model.
3. Proses mengubahnya menjadi format yang lebih cocok untuk memproses model disebut langkah transformasi data. Misalnya, mengubah data kategorikal menjadi data numerik atau kode yang tidak dapat diinterpretasikan oleh model pada kasus kapasitas ram yang telah dilakukan.
4. Proses mengubah data scaler ke rentang yang sama dikenal sebagai langkah penskalaan data. Ini sangat penting untuk mendapatkan hasil terbaik dengan model machine learning. 

Data preprocessing adalah langkah penting dalam machine learning yang menyiapkan data berkualitas tinggi untuk pelatihan model.  Metode ini dapat meningkatkan performa model dan memberikan hasil yang lebih presisi.

In [61]:
# Footer
myDate = datetime.datetime.now()
print("I certify that this is my own work.")
print("Signed by:")
print("Name: \t{}".format(myName))
print("NIM: \t{}".format(myNIM))
print("Time-stamp:\t{}".format(myDate))

I certify that this is my own work.
Signed by:
Name: 	Gempar Bambang Godata
NIM: 	00000067643
Time-stamp:	2023-02-17 05:13:16.482114


***

### Save the notebook, then convert the notebook to html (by running the next code).

In [62]:
!jupyter nbconvert --to html "./Template Laporan Tugas Mingguan Lab IF540.ipynb" --output-dir="./"

[NbConvertApp] Converting notebook ./Template Laporan Tugas Mingguan Lab IF540.ipynb to html
[NbConvertApp] Writing 587760 bytes to Template Laporan Tugas Mingguan Lab IF540.html


### Next step:
* convert the generated html file to PDF using the online tool: https://www.sejda.com/html-to-pdf
* choose the following settings:
    * Page size: One long page
    * Page Orientation: auto
    * Use print stylesheet
* Submit your ipython notebook and PDF files

Markdown basics https://markdown-guide.readthedocs.io/en/latest/basics.html#